In [8]:
import torch
import torch.nn as nn
import numpy as np
from customconv2d import conv2d as custom_conv2d

class CustomConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(CustomConv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))

    def forward(self, x):
        N, C, H, W = x.size()
        kernel_size = self.kernel_size
        stride = self.stride
        padding = self.padding

        output_height = (H + 2 * padding - kernel_size) // stride + 1
        output_width = (W + 2 * padding - kernel_size) // stride + 1
        output = torch.zeros((N, self.out_channels, output_height, output_width), device=x.device)

        for i in range(N):
            for j in range(self.out_channels):
                for k in range(C):
                    output[i, j] += torch.tensor(
                        custom_conv2d(
                            x[i, k].cpu().numpy(),
                            self.weight[j, k].cpu().numpy(),
                            stride=stride,
                            padding=padding
                        ),
                        device=x.device
                    )

        return output

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = CustomConv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = CustomConv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)

        return out

class SimpleResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = CustomConv2d(3, 64, kernel_size=7, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, 64, 2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks):
        layers = []
        for _ in range(blocks):
            layers.append(BasicBlock(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [11]:
import torch
model = SimpleResNet(num_classes=10)
input_tensor = torch.randn(1, 3, 224, 224)

model.eval()
with torch.no_grad():
    output = model(input_tensor)

print("Model output shape:", output.shape)

Model output shape: torch.Size([1, 10])
